## Data reprojection
- We want in this code to reproject a list of files with the corresponding desired reprojection
- We have to know what kind of data do we have for the data interpolation
- Once we have that, we can start reprojecting the data

In [1]:
"""This script goes for every tif file in a folder"""

import os
from sys import path
from osgeo import gdal
from numpy import random

In [ ]:
"""BEWARE WITH THE INDEXATION"""
# path=r"//akif.internal/public/z_resources/"
# out=r"//akif.internal/public/z_resources/"

path = r"C:/Users/admin/Documents/03_data/Urdaibai_data_for_server/raster"
out = r"C:/Users/admin/Documents/03_data/Urdaibai_data_for_server/raster/out"


"""
Tenemos que comprobar que los datos sean Categoricos o contiugos
Para categoricos habria que hacer el vecino proximo
Para continuos habría que hacerlo Bicubica - Bilineal
"""

os.chdir(r"C:/Users/admin/Documents/03_data/Urdaibai_data_for_server/raster")


# print(os.listdir(path))
#This iterates for every file in the directory
for file in os.listdir(path):
    #we only take the ones ended with tif
    if file.endswith('.tif'):
        #lets start analizing the data properties
        dataset = gdal.Open(file)
        stats =  dataset.GetRasterBand(1).GetStatistics(0,1) #it will return a list(Min, Max, Mean, StdDev)
        Metadata_domain = dataset.GetMetadataDomainList() #From here we get the 'IMAGE_STRUCTURE'
        image_structure = dataset.GetMetadata('IMAGE_STRUCTURE') #here inside

        # Use dict.get method in case the metadata dict does not have a 'COMPRESSION' key
        compression = image_structure.get('COMPRESSION', None)

        #Get a subset of the data as a test


        dataset = None
        # resampleAlg = gdal.GRIORA_NearestNeighbour
        # resampleAlg = gdal.GRIORA_Bilinear
        # esto añadelo al final

        #we concatenate the path
        input = path +"/"+ file
        output = out + '/' + file.replace(".tif","") + "_4326.tif"
        #here is the GDAL transformation, we set the path + / + variable
        gdal.Warp(output, input, creationOptions = ['COMPRESS=' + str(compression)], dstSRS = "EPSG:4326")
        
    else:
        pass


print("It's over the transformation")



The idea here was to see what kind of data pixels do we have, and in order of what we get, do a transformation in with the nearest neightbour,
bilineal, cubical or so... IT IS NOT FINISHED

In [ ]:
#We want to see here the inside of a raster file
os.chdir(r"C:/Users/admin/Documents/03_data/Urdaibai_data_for_server/raster/out")
file = "Arcilla_percentage_Urdaibai_4326.tif"

datafile = gdal.Open(file)
gt = datafile.GetGeoTransform() #we get the extreme values

band = datafile.GetRasterBand(1)

"""Here we get the size of the file"""
width = datafile.RasterXSize #columns
height = datafile.RasterYSize #rows
# define your tile size
# it could be 256,512 or 1024. it depends on your scope
tilesize = 50 #int((width * height) / 10)
"""Here we start with the tyiling"""
for i in range(0, width, tilesize): #tilesize marks from where to where in width
    for j in range(0, height, tilesize): #tilesize in height
        #for the edge parts, so we don't get nodata from the borders
        w = min(i+tilesize, width) - i
        h = min(j+tilesize, height) - j
        #we preapre the sentence
        array = band.ReadAsArray(j, i, w, h)
        
        print(array)
# datafile = None